# 🎬 Producción de Contenido Automatizada

Flujo completo: Análisis de transcripción → Edición de video → Multiplicación de contenido

**Pasos:**
1. Configurar entorno e instalar dependencias
2. Conectar Google Drive
3. **PASO 1**: Analizar transcripción y crear materiales
4. **PASO 2**: Editar video (cortes, audio, limpieza)
5. Multiplicar contenido (múltiples ratios y plataformas)


## 1️⃣ Instalación y Configuración


In [ ]:
# Instalar dependencias
%pip install -q faster-whisper torch torchaudio moviepy ffmpeg-python pydub


In [ ]:
# Configurar rutas y clonar/actualizar repo
import os
import sys
from pathlib import Path

# Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# Clonar o actualizar el repo
REPO_URL = "https://github.com/tu-usuario/contenido-automatico.git"  # Cambiar por tu repo
PROJECT_ROOT = Path("/content/contenido-automatico")

if not PROJECT_ROOT.exists():
    print("[*] Clonando repositorio...")
    !git clone {REPO_URL} {PROJECT_ROOT}
else:
    print("[*] Actualizando repositorio...")
    !cd {PROJECT_ROOT} && git pull

# Agregar al path
sys.path.insert(0, str(PROJECT_ROOT))

# Crear carpetas necesarias
(PROJECT_ROOT / "output").mkdir(exist_ok=True)
(PROJECT_ROOT / "samples").mkdir(exist_ok=True)

print(f"✅ Proyecto configurado en: {PROJECT_ROOT}")


In [ ]:
# Verificar GPU
from src.utils import verificar_entorno
info = verificar_entorno()


## 2️⃣ Subir Video de Entrada

Puedes subir el video de dos formas:
- Desde Google Drive (recomendado)
- Subir directo a Colab


In [ ]:
# Opción A: Usar video desde Google Drive
VIDEO_PATH = "/content/drive/MyDrive/videos/mi_video.mp4"  # Cambiar por tu ruta

# Opción B: Subir archivo directo
# from google.colab import files
# uploaded = files.upload()
# VIDEO_PATH = list(uploaded.keys())[0]

VIDEO_PATH = Path(VIDEO_PATH)
print(f"📹 Video seleccionado: {VIDEO_PATH}")
print(f"   Existe: {VIDEO_PATH.exists()}")


---
# PASO 1: Análisis y Transcripción

Recibe video y transcripción, analiza qué es útil y qué falta.


### 1.1 Extraer Audio y Transcribir


In [ ]:
from src.edicion import extraer_audio
from src.transcripcion import transcribir_audio

# Extraer audio
print("[*] Extrayendo audio...")
audio_path = extraer_audio(VIDEO_PATH, output_path=PROJECT_ROOT / "output" / "audio.wav")
print(f"✅ Audio extraído: {audio_path}")

# Transcribir
print("\n[*] Transcribiendo... (esto puede tardar varios minutos)")
resultado = transcribir_audio(audio_path, model_size="base")

palabras = resultado['palabras']
idioma = resultado['idioma']
texto_completo = resultado['texto_completo']

print(f"\n✅ Transcripción completada")
print(f"   Idioma: {idioma}")
print(f"   Palabras: {len(palabras)}")
print(f"\n📝 Primeras palabras: {texto_completo[:200]}...")


### 1.2 Analizar Transcripción


In [ ]:
from src.analisis import analizar_transcripcion

analisis = analizar_transcripcion(palabras)

print("=" * 60)
print("📊 ANÁLISIS DE TRANSCRIPCIÓN")
print("=" * 60)
print(f"Duración total: {analisis['duracion_total']:.2f} segundos")
print(f"Número de palabras: {analisis['num_palabras']}")
print(f"Silencios detectados: {len(analisis['segmentos_silencio'])}")

if analisis['segmentos_silencio']:
    print("\n⏸️ Silencios largos (>1.5s):")
    for idx, silencio in enumerate(analisis['segmentos_silencio'][:5], 1):
        print(f"  {idx}. {silencio['inicio']:.2f}s - {silencio['fin']:.2f}s ({silencio['duracion']:.2f}s)")


### 1.3 Ver Transcripción Completa


In [ ]:
# Guardar transcripción completa
transcript_path = PROJECT_ROOT / "output" / "transcripcion.txt"
with open(transcript_path, 'w', encoding='utf-8') as f:
    f.write(texto_completo)

print(f"✅ Transcripción guardada en: {transcript_path}")
print("\n" + "="*60)
print("TRANSCRIPCIÓN COMPLETA")
print("="*60)
print(texto_completo)


### 1.4 Crear Shotlist Manual

Basándote en la transcripción, define los cortes que quieres hacer.

**Formato:** `(inicio_segundos, fin_segundos)`


In [ ]:
from src.analisis import crear_shotlist

# Ejemplo: Definir cortes manualmente
# Cambia estos valores según tu análisis
CORTES_MANUALES = [
    (10, 30),   # Clip 1: intro
    (35, 60),   # Clip 2: punto principal
    (65, 90),   # Clip 3: conclusión
]

# Si quieres usar TODO el video, deja esta lista vacía
# CORTES_MANUALES = []

shotlist = crear_shotlist(palabras, cortes=CORTES_MANUALES)

print("✂️ SHOTLIST CREADA")
print("="*80)
for clip in shotlist:
    print(f"Clip {clip['orden']}: {clip['inicio']:.2f}s - {clip['fin']:.2f}s ({clip['duracion']:.2f}s)")
    print(f"  Texto: {clip['texto'][:100]}...")
    print()


📋 **PAUSA AQUÍ: Trabajo Manual**

En este punto debes:
1. Revisar la shotlist arriba
2. Crear/recopilar materiales extras (gráficos, overlays, música)
3. Ajustar los `CORTES_MANUALES` si es necesario
4. Cuando estés listo, continúa con el Paso 2


---
# PASO 2: Edición de Video

Cortar clips, limpiar audio, normalizar volumen, exportar.


### 2.1 Cortar y Concatenar Clips


In [ ]:
from src.edicion import cortar_video, concatenar_clips

# Cortar cada clip según la shotlist
clips_paths = []
output_clips_dir = PROJECT_ROOT / "output" / "clips"
output_clips_dir.mkdir(exist_ok=True)

print("✂️ Cortando clips...")
for clip in shotlist:
    clip_path = output_clips_dir / f"clip_{clip['orden']:02d}.mp4"
    print(f"  Clip {clip['orden']}: {clip['inicio']:.2f}s - {clip['fin']:.2f}s")
    
    cortar_video(
        VIDEO_PATH,
        inicio=clip['inicio'],
        fin=clip['fin'],
        output_path=clip_path
    )
    clips_paths.append(clip_path)

print(f"\n✅ {len(clips_paths)} clips cortados")

# Concatenar si hay múltiples clips
if len(clips_paths) > 1:
    print("\n[*] Concatenando clips...")
    video_editado = PROJECT_ROOT / "output" / "video_editado.mp4"
    concatenar_clips(clips_paths, video_editado)
    print(f"✅ Video editado: {video_editado}")
else:
    video_editado = clips_paths[0]
    print(f"✅ Video editado (clip único): {video_editado}")


### 2.2 Limpieza de Audio


In [ ]:
from src.edicion import reducir_ruido_basico

print("[*] Reduciendo ruido de audio...")
video_limpio = PROJECT_ROOT / "output" / "video_limpio.mp4"
reducir_ruido_basico(video_editado, video_limpio)
print(f"✅ Audio limpio: {video_limpio}")


### 2.3 Normalizar Audio


In [ ]:
from src.edicion import normalizar_audio_video

print("[*] Normalizando loudness a -14 LUFS...")
video_master = PROJECT_ROOT / "output" / "video_master.mp4"
normalizar_audio_video(video_limpio, video_master, target_lufs=-14)
print(f"✅ Video master listo: {video_master}")


### 2.4 Preview del Video Master


In [ ]:
from IPython.display import Video as VideoPlayer

print("🎬 VIDEO MASTER:")
VideoPlayer(str(video_master), width=800)


---
# PASO 3: Multiplicación de Contenido

Generar variantes para diferentes plataformas.


### 3.1 Exportar Múltiples Ratios


In [ ]:
from src.edicion import export_multi_ratio

# Definir qué ratios generar
RATIOS = ['16:9', '9:16', '1:1']  # YouTube, TikTok/Reels, Instagram

output_variantes = PROJECT_ROOT / "output" / "variantes"

print("[*] Generando variantes de aspect ratio...")
variantes = export_multi_ratio(video_master, output_variantes, ratios=RATIOS)

print("\n✅ VARIANTES GENERADAS:")
for ratio, path in variantes.items():
    print(f"  {ratio}: {path}")


### 3.2 Descargar Resultados


In [ ]:
# Opción 1: Copiar a Google Drive
import shutil

drive_output = Path("/content/drive/MyDrive/contenido_output")
drive_output.mkdir(exist_ok=True)

print("[*] Copiando resultados a Google Drive...")
shutil.copy2(video_master, drive_output / video_master.name)

for ratio, path in variantes.items():
    shutil.copy2(path, drive_output / path.name)

print(f"\n✅ Resultados copiados a: {drive_output}")


In [ ]:
# Opción 2: Descargar directo desde Colab
from google.colab import files

# Descomentar las líneas que quieras descargar
# files.download(str(video_master))
# for ratio, path in variantes.items():
#     files.download(str(path))


---
## ✅ Proceso Completo

Has completado:
1. ✅ Análisis de transcripción
2. ✅ Creación de shotlist
3. ✅ Edición de video (cortes, limpieza, normalización)
4. ✅ Multiplicación de contenido (múltiples ratios)

Los archivos están listos para:
- 🎬 YouTube (16:9)
- 📱 TikTok/Reels (9:16)
- 📸 Instagram (1:1)

**Próximos pasos manuales:**
- Agregar subtítulos (hardcoded o softcoded)
- Agregar overlays y gráficos
- Thumbnail y metadata
- Publicar en plataformas
